Weer data

In [5]:
import requests
import csv
import os
from datetime import datetime, timedelta
import pandas as pd

In [2]:
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": 51.05,
    "longitude": 3.7167,
    "start_date": "2016-10-01",
    "end_date": "2025-04-10",
    "hourly": "temperature_2m,apparent_temperature,rain,snowfall,weather_code,cloud_cover,wind_speed_10m,sunshine_duration",
    "timezone": "auto"
}

In [4]:
response = requests.get(url, params=params)

if response.status_code == 200:
    data = response.json()
    hourly_data = data.get("hourly", {})
    script_dir = os.getcwd()
    csv_file_path = os.path.join(script_dir, ".", "data csv", "weerdataraw.csv")
    os.makedirs(os.path.dirname(csv_file_path), exist_ok=True)

    #open csv
    with open(csv_file_path, mode="w", newline="") as file:
        writer = csv.writer(file)

        #header csv
        header = ["timestamp", "temperature_2m", "apparent_temperature", "rain", "snowfall", "weather_code", "cloud_cover", "wind_speed_10m", "sunshine_duration"]
        writer.writerow(header)

        #naar csv schrijven
        for i in range(len(hourly_data.get("temperature_2m", []))):
            row = [
                hourly_data.get("time", [])[i],
                hourly_data.get("temperature_2m", [])[i],
                hourly_data.get("apparent_temperature", [])[i],
                hourly_data.get("rain", [])[i],
                hourly_data.get("snowfall", [])[i],
                hourly_data.get("weather_code", [])[i],
                hourly_data.get("cloud_cover", [])[i],
                hourly_data.get("wind_speed_10m", [])[i],
                hourly_data.get("sunshine_duration", [])[i]
            ]
            writer.writerow(row)

    print(f"Data weggeschreven: {csv_file_path}")
else:
    print(f"Failed status code: {response.status_code}")

Data weggeschreven: c:\Users\krist\Documents\Hogent_IT\2de_jaar\Machine_Learning\ML_project_kijkcijfers\.\data csv\weerdataraw.csv


Kijkcijfer data

In [7]:
start_date = datetime(2016, 10, 1)
end_date = datetime.today()

script_dir = os.getcwd()
output_file = os.path.join(script_dir, "./data csv/kijkcijfersdataraw.csv")

os.makedirs(os.path.dirname(output_file), exist_ok=True)

In [8]:

with open(output_file, mode='w', newline='', encoding='utf-8-sig') as file:
    fieldnames = [
    'dateDiff', 'ranking', 'description', 'channel', 
    'startTime', 'rLength', 'rateInK', 'live'
    ]
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()

    # Loop door elke dag
    current_date = start_date
    while current_date <= end_date:
        datum = f"{current_date.year}-{current_date.month}-{current_date.day}"
        url = f"https://api.cim.be/api/cim_tv_public_results_daily_views?dateDiff={datum}&reportType=north"
            
        try:
            response = requests.get(url)
            if response.status_code == 200:
                data = response.json()
                programma_lijst = data.get('hydra:member', [])
                        
                for programma in programma_lijst:
                    try:
                        writer.writerow({
                        'dateDiff': programma.get('dateDiff'),
                        'ranking': programma.get('ranking'),
                        'description': programma.get('description'),
                        'channel': programma.get('channel'),
                        'startTime': programma.get('startTime'),
                        'rLength': programma.get('rLength'),
                        'rateInK': programma.get('rateInK'),
                        'live': programma.get('live')
                        })
                                    
                    except Exception as e:
                        print(f"error {datum}: {e}")         
            else:
                print(f"no data {datum}")
                        
        except Exception as e:
            print(f"error {datum}: {e}")
            
        current_date += timedelta(days=1)
print("data opgehaald")

data opgehaald
